---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

In [2]:
!head -5 Employee_Movie_Choices.txt

#Employee	Movie
Andy	Anaconda
Andy	Mean Girls
Andy	The Matrix
Claude	Anaconda


In [3]:
!head -5 Employee_Relationships.txt

Andy	Claude	0
Andy	Frida	20
Andy	Georgia	-10
Andy	Joan	30
Andy	Lee	-10


### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [4]:
def answer_one():
        
    # Your Code Here
    B = nx.read_edgelist('Employee_Movie_Choices.txt', delimiter="\t")
    
#     X, Y = bipartite.sets(B) ### separate the two sets
    pos= dict()
    pos.update((n, (1, i)) for i, n in enumerate(employees)) 
    pos.update((n, (2, i)) for i , n in enumerate(movies))
#     nx.draw(B,  pos = pos)
    nx.draw_networkx(B,  pos = pos)
#     plot_graph(B)
    return B# Your Answer Here


In [5]:
B = answer_one()

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [6]:
def answer_two():
    
    # Your Code Here
#     X, Y = bipartite.sets(B)
    for n in B.nodes():
        if n in movies:
            B.add_node(n, type='movie')
        elif n in employees:
            B.add_node(n, type='employee')
    return B# Your Answer Here

In [7]:
B=answer_two()

In [8]:
B.nodes(data=True)

[('Vincent', {'type': 'employee'}),
 ('Andy', {'type': 'employee'}),
 ('Anaconda', {'type': 'movie'}),
 ('Pablo', {'type': 'employee'}),
 ('Mean Girls', {'type': 'movie'}),
 ('Forrest Gump', {'type': 'movie'}),
 ('Joan', {'type': 'employee'}),
 ('Kung Fu Panda', {'type': 'movie'}),
 ('Frida', {'type': 'employee'}),
 ('The Godfather', {'type': 'movie'}),
 ('Monty Python and the Holy Grail', {'type': 'movie'}),
 ('The Social Network', {'type': 'movie'}),
 ('The Matrix', {'type': 'movie'}),
 ('The Dark Knight', {'type': 'movie'}),
 ('Lee', {'type': 'employee'}),
 ('Georgia', {'type': 'employee'}),
 ('Snakes on a Plane', {'type': 'movie'}),
 ('Claude', {'type': 'employee'}),
 ('The Shawshank Redemption', {'type': 'movie'})]

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [9]:
def answer_three():
        
    # Your Code Here
#     X, Y =  bipartite.sets(B)
    weighted_proj_B = bipartite.weighted_projected_graph(B, employees)
    return weighted_proj_B# Your Answer Here

In [10]:
P = answer_three()
P.edges(data = True)

[('Vincent', 'Pablo', {'weight': 1}),
 ('Vincent', 'Frida', {'weight': 2}),
 ('Andy', 'Frida', {'weight': 1}),
 ('Andy', 'Pablo', {'weight': 1}),
 ('Andy', 'Joan', {'weight': 1}),
 ('Andy', 'Georgia', {'weight': 1}),
 ('Andy', 'Claude', {'weight': 1}),
 ('Andy', 'Lee', {'weight': 1}),
 ('Claude', 'Georgia', {'weight': 3}),
 ('Pablo', 'Frida', {'weight': 2}),
 ('Joan', 'Lee', {'weight': 3})]

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [26]:
def answer_four():
    ### read in nextworks from Q3
    common_movie_df = pd.DataFrame(P.edges(data=True), columns = [0, 1,'#common_movies'])
    common_movie_df["#common_movies"] = common_movie_df['#common_movies'].map(lambda x: x['weight'])
    common_movie_df
    
    ### create a duplicate of dataframe, since the df is undirected graph
    common_movie_df2 = common_movie_df[[1, 0, "#common_movies"]]
    common_movie_df2.columns = [0,1,'#common_movies']
    common_movie_df2
    
    ### concat two dataframe to get an updated frame for both directions
    common_movie_df = pd.concat([common_movie_df,common_movie_df2])
    common_movie_df
    
    ### read in relations as df
    relation_df = pd.read_csv('Employee_Relationships.txt', delimiter="\t",header=None, names= [0,1, "Relations"])
    relation_df
    
    ### left join the relation_df witht he common_book_df to get the final dataframe
    merged = relation_df.merge(common_movie_df, on=[0,1], how='left')
    merged.fillna(0, inplace=True)
    merged.drop_duplicates(inplace=True)
    merged.reset_index(drop = True)

    return merged.corr().Relations[1] ### return the correlation between the relations and common book val using df.corr()

In [27]:
answer_four()

0.78839622217334759